# Регрессия

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings('ignore')

**Сначала попробуем решить задачу регрессии, то есть нашей целевой переменной будет 'index' - значение индекса человеческого развития**

Чтобы было честно, мы уберем из выборки признак classification, то есть другую нашу целевую переменную, предсказывать которую мы будем в процессе решения задачи классификации.


In [3]:
df = pd.read_csv('df_vib.csv')

#### Для начала, приведём целевую переменную на промежуток от 0 до 100, это сделано для того чтобы при возведении в квадрат мы не получали слишком маленькие цифры

In [4]:
df['index'] = df['index']*100
df.head(2)

,YEAR,DISORDER_TYPE,EVENT_TYPE,ACTOR1,INTER1,ACTOR2,INTER2,INTERACTION,CIVILIAN_TARGETING,COUNTRY,LOCATION,LATITUDE,LONGITUDE,NOTES,FATALITIES,index,classification,bool,violence_ind
0,2022,Demonstrations,Protests,Protesters (Italy),6,NaN,0,60,NaN,Italy,Cagliari,39.217,9.113,"On 15 March 2022, striking haulers from across...",0,89.5,High income,0,1.0
1,2020,Demonstrations,Protests,Protesters (United States),6,NaN,0,60,NaN,United States,Chicago,41.850,-87.650,"On 5 October 2020, student-athletes staged a p...",0,92.0,High income,0,1.0


#### Заполним пропуски $0$ и удалим столбцы, не информативные для предсказания

In [5]:
df_linear_model = df.fillna(0)
df_linear_model = df_linear_model.drop(['ACTOR1', 'ACTOR2', 'INTER1', 'INTER2', 'COUNTRY', 'LOCATION', 'NOTES','CIVILIAN_TARGETING', 'LATITUDE', 'LONGITUDE', 'classification','DISORDER_TYPE'], axis = 1)
df_linear_model.head(2)

,YEAR,EVENT_TYPE,INTERACTION,FATALITIES,index,bool,violence_ind
0,2022,Protests,60,0,89.5,0,1.0
1,2020,Protests,60,0,92.0,0,1.0


#### Очевидно, что нам нужно закодировать категориальные переменные. Воспользуемся для этого one-hot кодированием

In [6]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(drop='first')
encoder_df = pd.DataFrame(encoder.fit_transform(df_linear_model[['EVENT_TYPE']]).toarray())
df_linear_model = df_linear_model.join(encoder_df).drop(['EVENT_TYPE'],axis = 1)

df_linear_model = df_linear_model.rename(columns={0:"Protests", 1: "Strategic developments", 2: 'Battles', 3: 'Riots', 4:'Violence against civilians'})

In [7]:
df_linear_model.head(2)

,YEAR,INTERACTION,FATALITIES,index,bool,violence_ind,Protests,Strategic developments,Battles,Riots,Violence against civilians
0,2022,60,0,89.5,0,1.0,0.0,1.0,0.0,0.0,0.0
1,2020,60,0,92.0,0,1.0,0.0,1.0,0.0,0.0,0.0


#### Ну и, конечно же, разделим выборку на обучающую и тестовую, с долей последней в 20%

In [8]:
from sklearn.model_selection import train_test_split
np.random.seed(321)
x_train, x_test = train_test_split(df_linear_model, test_size = 0.2)

**Начнем с базы, с голой линейной регрессии**

In [9]:
y_train, y_test = x_train['index'], x_test['index']
x_train, x_test= np.array(x_train.drop('index', axis=1)), np.array(x_test.drop('index', axis=1))

In [10]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)

In [11]:
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse

print(f'MSE: {mse(y_test, y_pred)}')
print(f'MAE: {mae(y_test, y_pred)}')

MSE: 163.44426922919263
MAE: 10.982323276578061


#### Как видно, даже без подбора гиперпараметров, наша модель дала небольшие значения метрик MSE и MAE

#### Давайте также посмотрим на коэффициенты, которые модель присвоила имеющимся признакам

In [12]:
model.coef_

array([ 1.43681698, -0.01339989,  0.14146283, -2.18107901, -0.27467269,
       -1.98867265,  4.59585018,  3.52699952, -2.41477117,  0.20036953])

Самый большой коэффициен у признака Protests, можно сказать, что он вносит наибольший вклад в обучение нашей модели, что согласуется с нашим предположением о большем развитии стран, где превалируют ненасильственные формы конфликтов

**Теперь попробуем обучить линейную модель с регуляризацией, но уже отнормировав признаки**

In [13]:
np.random.seed(321)
x_train, x_test = train_test_split(df_linear_model, test_size = 0.2)
y_train, y_test = x_train['index'], x_test['index']
x_train, x_test= x_train.drop('index', axis=1), x_test.drop('index', axis=1)
x_train.head(2)

,YEAR,INTERACTION,FATALITIES,bool,violence_ind,Protests,Strategic developments,Battles,Riots,Violence against civilians
9471,2009,37,1,1,21.1,0.0,0.0,0.0,0.0,1.0
791,2017,60,0,0,1.0,0.0,1.0,0.0,0.0,0.0


#### Отмасштабируем признаки

In [14]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train = np.hstack((x_train.drop(['YEAR',	'INTERACTION',	'FATALITIES',	'bool',	'violence_ind'],axis = 1), scaler.fit_transform(x_train[['YEAR',	'INTERACTION',	'FATALITIES',	'bool',	'violence_ind']])))
x_test = np.hstack((x_test.drop(['YEAR',	'INTERACTION',	'FATALITIES',	'bool',	'violence_ind'],axis = 1), scaler.fit_transform(x_test[['YEAR',	'INTERACTION',	'FATALITIES',	'bool',	'violence_ind']])))

#### Попробуем минимизировать MAE

In [19]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge
model = Ridge(alpha=1)
param_grid = {
    'alpha': np.arange(1, 1000)

}
grid_search = GridSearchCV(model, param_grid, cv=4, scoring = 'neg_mean_absolute_error')

grid_search.fit(x_train, y_train)

grid_search.best_estimator_

Ridge(alpha=9)

In [20]:

model = Ridge(alpha=9, solver='auto')
model.fit(x_train, y_train)
y_pred = model.predict(x_test)

print(f'MSE: {mse(y_test, y_pred)}')
print(f'MAE: {mae(y_test, y_pred)}')

MSE: 163.71421555797383
MAE: 10.9881727809614


Как видно, нам не удалось добиться  снижения mae, ведь наша первая модель уже выдала хороший результат

**Теперь попробуем использовать случайный лес**

In [21]:
np.random.seed(321)
x_train, x_test = train_test_split(df_linear_model, test_size = 0.2)
y_train, y_test = x_train['index'], x_test['index']
x_train, x_test= x_train.drop('index', axis=1), x_test.drop('index', axis=1)
scaler = StandardScaler()
x_train = np.hstack((x_train.drop(['YEAR',	'INTERACTION',	'FATALITIES',	'bool',	'violence_ind'],axis = 1), scaler.fit_transform(x_train[['YEAR',	'INTERACTION',	'FATALITIES',	'bool',	'violence_ind']])))
x_test = np.hstack((x_test.drop(['YEAR',	'INTERACTION',	'FATALITIES',	'bool',	'violence_ind'],axis = 1), scaler.fit_transform(x_test[['YEAR',	'INTERACTION',	'FATALITIES',	'bool',	'violence_ind']])))

In [26]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators=1, max_depth=1, min_samples_split=1, min_samples_leaf=1, max_features=1, random_state=321)
param_grid = {
    'n_estimators': [55, 60, 70, 80, 90],
    'max_depth' : [10 , 25, 30, 35],
    'min_samples_split': [1 , 2 , 10 , 20],
    'min_samples_leaf': [10, 15, 20],
    'max_features': [6,8, 10]
}
grid_search = GridSearchCV(model, param_grid, cv=2, scoring = 'neg_mean_absolute_error')

grid_search.fit(x_train, y_train)

grid_search.best_estimator_

RandomForestRegressor(max_depth=25, max_features=10, min_samples_leaf=10,
                      n_estimators=60, random_state=321)

In [35]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(max_depth=25, max_features=2, min_samples_leaf=5,n_estimators=150, random_state=321)
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
print(f'MSE: {mse(y_test, y_pred)}')
print(f'MAE: {mae(y_test, y_pred)}')

MSE: 148.80802917368203
MAE: 10.110323390242854


Случайный лес показал себя куда лучше, MSE упал почти на 20%

**Как видно, мы получили сравнительно хорошие значения для наших функций потерь, а важноссть признаков согласуется с нашей основным предположением**